# Course Two: Data Preparation

Expected time to complete: 45 minutes

## Goal of this Course

In this course we will transform our dataset into a format that can be used with a statistical model. We will first explain some key concepts that will help us to understand in which scope our problem stands.  

The course is split into the following parts:
- <a href='#context'>Context</a> 
- <a href='#supervised_vs_unsupervised'>Supervised vs Unsupervised Learning</a>
- <a href='#classif_vs_reg'>Classification vs Regression</a>
- <a href='#model_output'>Model Output</a> 
    - <a href='#load_data'>Load the Data</a>
    - <a href='#define_target'>Define our Target</a>
    - <a href='#basic_statistics'>Descriptive Statistics</a>


- <a href='#model_input'>Model Input</a> 
    - <a href='#feature_selection'>Feature Selection</a>
    - <a href='#basic_statistics2'>Descriptive Statistics</a>


- <a href='#pair_input_output'>Pair Model Input with Model Output</a> 
- <a href='#missing_data'>Check for Missing Data</a> 

<a id='context'></a>
# Context

In the previous tutorial we formulated a data problem, chose a dataset and performed some initial steps that cleaned  and then helped us explore the structure of our data. In this second course we will start looking at how to _prepare_ our dataset to be compatible with the kind of statistical models that can help us solve our problem.

Data preparation or _data wrangling_ is a crucial step in the modelling process. It refers to how we create a dataset to be ingested into at least one of the many statistical models at our disposal. 

We need to make sure we understand the scope of the problem:
- Do we have a _target_ variable? In other words, will our model predict something (the target) or will it look for unlabeled structure? (supervised vs unsupervised models)
- If predicting, what form will the prediction target have? Is it represented as discrete categories or a linear scale (classification or regression)?
- If predicting, what is the granularity? User level? User x time level?
- What features of our dataset are we going to use as input to our model (feature selection)?

Before we dive back into the code, let's explain a few core concepts that will be helpful to understand before we go any further.

<a id='supervised_vs_unsupervised'></a>
# Supervised vs Unsupervised Learning

An important disctinction to make when preparing your data for analysis is whether your problem is suitable for “supervised” or “unsupervised” learning models. 
 
__Unsupervised learning__ models receive input (_explanatory/predictor/independent_ variables or _features_) but no output (or _dependent/target_ variable)  and are a way of discovering latent structure in a set of data (clustering is an example of unsupervised learning). Unsupervised models are very useful when working with unlabelled datasets. These models can then be (and often are) combined with supervised models.
 
__Supervised learning__ models learn a mathematical function between an input (_explanatory/predictor/independent_ variables) and an output (the _dependent/target_ variable). These models are used in situations where you know what you want to predict and have explicit input-output pairs for your model to be trained upon.

In our current project we want to forecast the minutes watched on iPlayer (output or _target_ variable) based on past behavior (input or _explanatory/predictor/independent_ variables). We have input-output pairs and are, therefore, in the supervised learning framework.

<a id='classif_vs_reg'></a>
# Classification vs Regression

Within the supervised learning class of problems there is a further split, based upon the nature of the target variable: discrete or linear. If a dataset is being used to predict one of a finite set of categories, this is called a classification problem and is tackled by a specific set of models called _classifiers_. A simple example of a classification problem is a spam email filter where the text of the email must be classified as either legitimate or not. A further exampleof a _multi-classification_ problem is handwriting recognition, where a hand-written letter is classified as one of a set of 36 (26 letters and 10 digits).

Alternatively, some data problems require the prediction of linear or non-categorical variables. This class of problems is approached using a class of model called _regressors_. An example of a regression problem would be to predict the number of hits an article receives by analysing the contents of its headline.

In our current project we want to predict if a user watched any content within a two-week period (a classification problem), and we also want to know _how much_ content a user watched within a two-week period (a regression problem). We will therefore be using both classifiers and regressors in this project.

<a id='model_output'></a>
# Model Output

<a id='load_data'></a>
## Load the Data
Now we read our data from the last course back in and check that it looks reasonable.

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 

In [ ]:
# Load our output dataset from course 1
data = pd.read_csv('iplayer_data_c1.csv')

# Check that it all looks how we expect it to.
data.head()

This should all look very familiar by now. When computing the distribution of our observations among the `twoweek` variable (Course 1 - _Exploratory data analysis_) we saw that we had roughly the same amount of data, except for week 0. Here, as it is not clear why it should have less data (meaning it may be a sign of something not being right), we decide to remove this group from all our subsequent analyses.

In [ ]:
data=data[data['twoweek']>0]

In [ ]:
data.twoweek.value_counts().sort_index()

<a id='define_target'></a>
## Define our Target

In this section we will walk through the process of how to create the output of our model. This variable that supervised models predict is know as the _target variable_, the _output variable_ or the _dependent variable_.

In our project we want to forecast the next two weeks behaviour, so we need to aggregate our target metric for the last two weeks of data available, i.e. how many minutes of content, if any, did each individual audience member consume within `twoweek` 8. 

Our target variable will be different for our two different types of model:

- In our _regression model_ our target is the total minutes watched within the final two week period.

- In our _classification model_  our target is a binary variable (0 or 1), where we will forecast whether a user will watch iPlayer or not (total minutes watched > 0) within the final two week period.


### Regression Model

In our specific problem, we want to forecast what _individual_ users will do. Therefore, we need to pivot our dataset from an _events view_ to a _user view_,  to build both our target and features.

To calculate minutes watched for each audience member we create a _pivot table_. This process results in a table where each individual audience member is represented as a row and each column describes the total minutes watched for a given two-week period. As all the audience members who consumed no content during a given two-week period are represented as "NA" after this operation, we replace all NAs with zeros. 

In [ ]:
# We need to compute the total minutes watched within twoweek 8 for each user
target_reg=pd.pivot_table(data,
                          values='min_watched',
                          index=['user_id'],
                          columns=['twoweek'],
                          aggfunc=sum)[8].reset_index()


# We fill the NAs with 0: corresponds to the users without observations for the last twoweek
# i.e. who didn't watch anything
target_reg=target_reg.fillna(0)
target_reg.head()

### Classification Model

To calcualate our target variable for the classification model we simply replace all numbers from the minutes watched column that are above zero with 1. 

In [ ]:
# We build the dummy variable based on the minutes watched on this twoweek
target_class=target_reg.copy()
target_class[8]=np.where(target_class[8]>0,1,0)
target_class.head()

<a id='basic_statistics'></a>
## Descriptive Statistics

Both for a quick consistency check - and to have business insights related to engagement - we will compute some descriptive statistics on our target variables.

### Regression Output - minutes watched
First, let's have a look at the basic statistics for the minutes watched per user in the final two-week group.

In [ ]:
# Basic statistics for a quantitative variable
target_reg.describe()

Here we can see that on average the users watched 76mn on `twoweek` 8. The minutes watched aren't equally distributed among the users though as the median is equal to 0 and the standard error quite huge.

Let's plot the histogram to have a better idea of the distribution.

In [ ]:
# Distribution of the minutes watched
target_reg[8].hist(bins=100)
plt.title('Distribution of target variable (total minutes watched in final two-week group)')
plt.show()

There are 100 bins and the scale reaches 10000 therefore each bar represents 100 minutes. This indicates the vast majority of people (represented as the first bar) consumed less than 100 minutes in the final two-week period.

### Classification Output - did the user watch iPlayer or not

Now let's have a look at the counts of the the classifier target variable. As expected (we see just zeros and ones) it appears that there are more viewers that did not consume any content in the final two-week period than did.

In [ ]:
# Distribution of the dummy variable did the user watch or not
target_class[8].value_counts().sort_index().plot(kind='bar')
plt.title('Distribution of target variable (total minutes watched in final two-week group)')
plt.show()

In [ ]:
# Ratio 
print("More precisely, "+
      str(round(sum(target_class[8])/len(target_class)*100,2))+
      "% of the users in our sample watched iPlayer within the last two-week period.")

<a id='model_input'></a>
# Model Input

In this section we will walk through the process of how to select and calculate input to our model. The input to a statistical model can often be confusing as it can be described in many different ways. _Independent variables, explanatory variables, predictor variables_ and _features_ are all ways of describing the same thing: the input to your statistical model.

<a id='feature_selection'></a>
## Feature Selection
As mentioned in the first course, in most situations the process of selecting our input or _feature engineering_ is an iterative one until you get the feature set that neither _underfits_ or _overfits_ the data.

A feature set that does not contain sufficient information about the output variable will often result in the model underfitting (this can usually be identified by a high _training error_). The solution here is often to add more features. If the feature set contains features that are sensitive to spurious and random elements of the dataset (and not the underlying population it should be an approximation of), _overfitting_ occurs. Overfitting is characterised by low _training error_ and high _test error_. Overfitting can be tackled by reducing the complexity of your model (often removing features) or using regularisation techniques (https://www.quora.com/What-is-regularization-in-machine-learning). A larger and more diverse training set also helps to reduce overfitting. We will come back to the concept of training and test errors in the following course.

There are various feature selection tools that can be used together with cross-validation to optimise your feature set (e.g. _stepwise regression_ - https://en.wikipedia.org/wiki/Stepwise_regression).  
 
In our project we choose to focus on features that describe the viewing habits of a particular user (e.g. “average completion”, “most watched genre”, ”time watched“).

### User Granularity
The input variables for regression and classifications models are no different here, so we will perform the same process for each. Just as for our target variable, our input variables must have a `user_id` granularity (i.e. each must be paired with a unique audience member).

N.B. As our dataset contains temporal properties, we could have performed a more granular pivot on `user_id x twoweek` (i.e. a user represented as seven rows - one for each two-week group). Splitting the dataset up in this way, allows certain types of model to exploit any temporal contingencies that may exist in the data. To keep things simple, here we stick to a single row per user.

In [ ]:
# Create a function that pivots the data based on customer
# and gives us all the features we need
def pivot_data(dataframe):
    #How many minutes did each person watch in each 2 week period
    data=pd.pivot_table(dataframe,values='min_watched', 
                        index=['user_id'],columns=['twoweek'], aggfunc=sum)
    # Fill the weeks they didn't watch in with 0s
    data.fillna(0,inplace=True)
    # How much of average did each viewer watch?
    data['average_completion']=dataframe.groupby('user_id')['percentage_watched'].mean()
    # How many sessions did the person have with us
    data['total_sessions']=dataframe.groupby('user_id')['streaming_id'].nunique()
    # How many times has the viewer watched something
    data['number_watched']=dataframe.groupby('user_id')['streaming_id'].count()
    # Genre most watched by the viewer
    data['most_genre']=pd.pivot_table(dataframe,values='min_watched', index=['user_id'],
                                      columns=['enriched_genre'], aggfunc=sum).idxmax(axis=1)
    # Number of genres watched
    data['num_genre']=pd.pivot_table(dataframe,values='min_watched', index=['user_id'],
                                     columns=['enriched_genre'], aggfunc=sum).count(axis=1)
    # Favourite day of the week to watch
    data['most_weekday']=pd.pivot_table(dataframe,values='min_watched', index=['user_id'],
                                        columns=['weekday'], aggfunc=sum).idxmax(axis=1)
    # Number of weekdays watched
    data['num_weekday']=pd.pivot_table(dataframe,values='min_watched', index=['user_id'],
                                       columns=['weekday'], aggfunc=sum).count(axis=1)
    # Favorite time of day to watch
    data['most_timeday']=pd.pivot_table(dataframe,values='min_watched', index=['user_id'],
                                        columns=['time_of_day'], aggfunc=sum).idxmax(axis=1)
    # Number of times of day
    data['num_timeday']=pd.pivot_table(dataframe,values='min_watched', index=['user_id'],
                                       columns=['time_of_day'], aggfunc=sum).count(axis=1)
    return data

In [ ]:
# We need to consider here only the "past data", i.e. get rid of the last twoweek obs
# which corresponds to our target
features=pivot_data(data[data['twoweek']<8])
features.reset_index().head()

So now we have a set of features for each user. It currently includes:
- the minutes watched on a two-week basis for the past 14 weeks `1`, `2`,..., `7`

And aggregated over this 14 weeks timeframe:
- `average_completion` - the average completion when watching a piece of content 
- `total_sessions` - the number of sessions 
- `number_watched` - the number of time a user watched something 
- `most_genre` - the main genre watched in terms of minutes and not number of pieces of content
- `num_genre` - the number of different genre watched
- `most_weekday` - the favourite day of the week to watch - again in minutes watched
- `num_weekday` - the number of differents days of the week a user watched something
- `most_timeday` - the favourite time of the day to watch - again in minutes watched
- `num_timeday` - the number of differents times of the day a user watched something

This set of variables constitute our features space to build the models on. Each row in this table (without the userID) is referred to as a _feature vector_.  Note that we could have imagined lots of other features.

### Dummification
Most models only take in quantitative data; data represented with numbers. We therefore need to convert those that contain strings to categorical numeric representations. This process is often referred to as creating _dummy variables_. To do this we will replace each unique string within a categorical variable with an aditional column which can contain either a zero or a one.  

In [ ]:
# Turn our categorical variables into bins so that we can run models on this
features=pd.get_dummies(features).reset_index()
features.head()

### Naming

Here we change the name of the columns representing the two-week minutes watched variables to make them generic:  `tw_lag7_watched`, `tw_lag5_watched`, ..., `tw_lag1_watched`.

If we want to use our model later for forecasting purposes on a new dataset with a new timeframe, this will make things much less confusing. 

In [ ]:
# Make the column names generic
features = features.rename(columns={1:'tw_lag7_watched',
                                    2:'tw_lag6_watched',
                                    3:'tw_lag5_watched',
                                    4:'tw_lag4_watched',
                                    5:'tw_lag3_watched',
                                    6:'tw_lag2_watched',
                                    7:'tw_lag1_watched'})
features.head()

<a id='basic_statistics2'></a>
## Descriptive Statistics

As for the output variables, we can compute some basic statistics for our input variables.

### Distributions
We will first visualise their distributions by plotting histograms. There are 35 features in all so going into details of all of them would take a lot of time, but you should notice that - broadly speaking - they should reflect the distributions we observed in the previous course.

#### Minutes watched per two-week group

In [ ]:
# Histogram 
for i in [x for x in list(features.columns) if 'tw_lag' in x]:
    features[i].hist(bins=50)
    plt.title(i)
    plt.show()

These distributions underline __different levels of engagement__ within the users on this two-week timeframe.


#### Total number of sessions over the entire timeframe

In [ ]:
# Histogram
features['total_sessions'].hist(bins=50)
plt.title('Total number of sessions per user')
plt.show()

#### Number of unique pieces of content watched over the entire timeframe

In [ ]:
# Histogram
features['number_watched'].hist(bins=50)
plt.title('Number of unique pieces of content watched per user')
plt.show()

#### Average completion over the entire timeframe

In [ ]:
# Histogram
features['average_completion'].hist(bins=50)
plt.title('Average completion per user')
plt.show()

In case you were wondering about the different shape of this distribution compared to the _Programme duration_ plot in the _Descriptive Statistics_ section in Course 1 

The first distribution (in course 1) is a representation of the individual views; this shows that overwhelmingly people turn off within the first 2 minutes or finish the whole thing (1 or 0).  The second distribution (above) is the average completion of an audience member. So if they registered two views: one completed, and one switched off after 10 seconds, they would register in this second distribution as 0.5 average completion.
So the skew towards zero is an indication that people start more shows than they complete.
The bumps at 1 and zero represent viewers that either complete nothing they start or complete everything they start.

#### Diversity in the consumption of iPlayer

We will have a look here at the distribution of the following variables:
- `num_genre` - number of different genres watched over the entire timeframe
- `num_weekday` - number of different days of the week the user watched iPlayer over the entire timeframe
- `num_timeday` - number of different time of the day the user watched iPlayer over the entire timeframe

In [ ]:
# Histogram 
for i in ['num_genre', 'num_weekday','num_timeday']:
    features[i].value_counts().sort_index().plot(kind='bar')
    plt.title(i)
    plt.show()

Remark: for few users these variables are null. These users only have observations for the last two-week group that we are not considering here. So we have null rows in our feature space. We will get rid of them later in the course. 

#### Favourite genre over the entire timeframe

In [ ]:
# Let's merge the different dummies related to the favourite genre
# in order to have only one histogram 
favourite_genre=dict()
for i in [x for x in list(features.columns) if 'most_genre' in x]:
    favourite_genre[i.split('most_genre_',1)[1]]=sum(features[i]) 
pd.Series(favourite_genre)

In [ ]:
# Histogram
pd.Series(favourite_genre).sort_index().plot(kind='bar')
plt.title('Favourite genre watched per user')
plt.show()

#### Favourite day of the week over the entire timeframe

In [ ]:
# Let's merge the different dummies related to the favourite day of week
# in order to have only one histogram 
favourite_weekday=dict()
for i in [x for x in list(features.columns) if 'most_weekday' in x]:
    favourite_weekday[i.split('most_weekday_',1)[1]]=sum(features[i]) 
pd.Series(favourite_weekday)

In [ ]:
# Histogram
pd.Series(favourite_weekday).sort_index().plot(kind='bar')
plt.title('Favourite day of the week to watch iPlayer per user')
plt.show()

#### Favourite time of the day over the entire timeframe

In [ ]:
# Let's merge the different dummies related to the favourite time of the day
# in order to have only one histogram 
favourite_timeday=dict()
for i in [x for x in list(features.columns) if 'most_timeday' in x]:
    favourite_timeday[i.split('most_timeday_',1)[1]]=sum(features[i]) 
pd.Series(favourite_timeday)

In [ ]:
# Histogram
pd.Series(favourite_timeday).sort_index().plot(kind='bar')
plt.title('Favourite time of the day to watch iPlayer per user')
plt.show()

As expected, we don't learn much more compared to the descriptive exploration of the previous course. Nervertheless, it is a mandatory consistency check and it can sometimes allow to spot outliers at the _user level_.

### Visualising the Relationships between Features

A _correlation matrix_ is a good way of getting a quick grasp of the relationships between our features. Correlation is a statistical test which measures the strength of a linear relationship between two variables. The correlation coefficient is a value between -1 and 1. Variables that are perfectly correlated or perfectly inversely-correlated  have coefficients of 1 and -1 respectively. Variables that have no linear relationship will have a coeffient of zero. A correlation matrix calculates coefficients for every pair of variable and illustrates them _n_ x _n_ grid (where n= the number of features). 

For many models (_regressions_ for example) we have to avoid multicolinearity in our features space. If input variables are correlated, some assumptions in the modelling framework won't hold anymore and it will result in inaccurate results. 

In [ ]:
# Correlation matrix
plt.matshow(features.corr())
plt.colorbar()
plt.show()

In [ ]:
# Reminder on the different input variables 
input_var=dict()
i=0
for x in features.columns:
    input_var[i]=x
    i+=1
input_var

In the correlation matrix of our input features above,  we can infer that there is a strong linear relationship between `total_sessions` and `number_watched`. We decided to remove `number_watched` for the rest of the analyses as it is highly correlated with other variables. We do this because because some of the models we will be using later in our analyses assume that our features are not dependent upon each other. This would be a violation of multicolinearity in regression models. Some other variables are highly correlated but not to that extent; so we will keep them. 

Correlation matrix is then useful in the feature selection process. It helps identify _redundancy_ between your features and therefore identify candidates for removal when needing to reduce complexity.  

In [ ]:
features=features.drop(['number_watched'], axis=1)

<a id='pair_input_output'></a>
# Pair Model Input with Model Output

Now we need to ensure that our input variables match up with our output variable. Each row in our input table should match up to the same `user_id` in our output table. If features or target values are missing for a given user we will get rid of this observation.

In [ ]:
# Find the unique users in both the features and the target
users_target=target_reg['user_id'].unique()
users_features=features['user_id'].unique()

# Find those users that are in the target but not in the feature
target_not_feature=[]
for user in users_target:
    if user not in users_features:
        target_not_feature.append(user)

# Find those users that are in the feature but not in the target
feature_not_target=[]
for user in users_features:
    if user not in users_target:
        feature_not_target.append(user)

# Print the size of the two sets
print('In target but not feature:',len(target_not_feature),
      '- In feature but not target:' ,len(feature_not_target))

NB: it's actually normal to have noone missing in the second case because we built the target variables based on the entire population of the training data. It's more a consistency check here.

We need to remove any users that don't have recorded viewing behaviour before `twoweek` 8. These users can be explained in two ways: either they did not consume any content in the first seven two-week groups or they only signed up to iPlayer in the final two-week group. Neither of these groups are useful from a modeling perspective so we will remove them all. 

NB: A `time_since_account_created` variable could be an informative feature to consider in a future model. However, we might need some _business logic_ to avoid what is often referred to as _left-censoring_ issues; i.e. as we can't usually observe the entire historic, it's often difficult to have a signing date for all users. We might have began the tracking of our data after the first user accounts.

In [ ]:
# We will set the index to the user_id as this will make it easier to drop rows
# Then we drop the rows and then turn the remaining column into an array
target_reg=target_reg.set_index(['user_id'])
target_reg.drop(target_not_feature,inplace=True)
target_reg.reset_index(inplace=True)
target_reg=target_reg[8].values

# Same for the classification
target_class=target_class.set_index(['user_id'])
target_class.drop(target_not_feature,inplace=True)
target_class.reset_index(inplace=True)
target_class=target_class[8].values
        
# Check to make sure the outcome makes sense
print(target_reg[:10])
print(target_class[:10])

In [ ]:
# Let's check the size of our datasets
print('Number of samples in the training feature set:',len(features))
print('Number of samples in the training target set (classification):',
      len(target_class))
print('Number of samples in the training target set (regression):',
      len(target_reg))

<a id='missing_data'></a>
# Check for Missing Data

The last thing we need to do is to replace any possible missing values with zeros and remove the `user_id` field from each row. This field contains useful information for indexing but it does not contain useful modeling information. Therefore, we shall move it from a normal column to an index column.

In [ ]:
# We will fill remaining missing values with 0s as we don't know any better
features=features.set_index(['user_id'])
features.fillna(0,inplace=True)
features.head()

 Now that we have our data in the perfect condition to ingest into a statistical model, we shall save it on disk so that we can continue working on it in the next course.

In [ ]:
features.to_csv('features.csv')
np.savetxt('target.txt',(target_reg,target_class))

# Summary

In this course we looked at the process of preparing our dataset to be ingested into a statistical model.

- We learned about how data should be prepared in different ways based upon the kind of problem you have (supervised or unsupervised) and the kind of model you intend to use (classification or regression).
- We learned that for supervised learning problems, input/output pairs need to be calculated from our original dataset. 
- We learned about useful tools such as the generation of pivot tables that helps translate our dataset from event-centered rows to user-centered rows.

In the next course we will start the modelling.